In [1]:
import numpy as np
import scipy.stats as sps
import scipy.integrate as spi

## Красивая табличка

| № | Содержание $Ig$ $A(\%)$                    |
|---|--------------------------------------------|
| 1 | 83, 85                                     |
| 2 | 84, 85, 85, 86, 86, 87                     |
| 3 | 86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90 |
| 4 | 89, 90, 90, 91                             |
| 5 | 90, 92                                     |

In [7]:
samples_ig = [
    [83, 85],
    [84, 85, 85, 86, 86, 87],
    [86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90],
    [89, 90, 90, 91],
    [90, 92]
]

Вместо показателей иммуноглобулина делаем группы возрастные и теперь у нас 5 факторов: $x_1$, $x_2$, $x_3$, $x_4$, $x_5$

In [8]:
sample = []
for i, sample_ig in enumerate(samples_ig):
    for _ in range(len(sample_ig)):
        sample.append(np.eye(5)[i])

sample = np.array(sample)
sample_sum = np.array([np.sum(group) for group in samples_ig])
sample_count = np.array([len(group) for group in samples_ig])

## Создаем вектор ответов

In [69]:
def flatten(array: list) -> list:
    result = []
    for element in array:
        result.extend(element)
    return result

In [70]:
Y = flatten(samples_ig)
Y = np.array(Y).reshape((1, -1)).T

In [71]:
class LinearRegression:

    def __init__(self, factors: np.array, answers: np.array):
        self.factors = factors
        self.answers = answers

        self.F = factors.T @ factors
        self.inv_F = np.linalg.inv(self.F)
        self.coefficients = self.inv_F @ factors.T @ answers
        self.errors = self.answers - self.factors @ self.coefficients

        self.RSS = (self.errors.T @ self.errors)[0, 0]
        self.TSS = np.sum((self.answers - np.average(self.answers)) ** 2)
        self.R_square = (self.TSS - self.RSS) / self.TSS

    def is_regression_significant(self):
        delta = self.R_square / (1 - self.R_square) * (
                    self.n - self.factors_count) / (self.factors_count - 1)
        p_value = spi.quad(sps.f.pdf, delta, np.inf, args=(
        self.factors_count - 1, self.n - self.factors_count))[0]
        return p_value < 0.05

    def check_multicollinearity(self):
        R_squares = []
        for i in range(0, self.factors_count):
            temp_factors = np.delete(self.factors, i, 1)
            temp_answers = self.factors[:, i]

            R_squares.append(
                LinearRegression(temp_factors, temp_answers).R_square)
        return R_squares

    def check_significant(self):
        p_values = []
        for i, coefficient in enumerate(self.coefficients):
            delta = coefficient / np.sqrt(
                self.RSS * self.inv_F[i][i]) * np.sqrt(
                self.n - self.factors_count)
            p_value = 2 * spi.quad(sps.t.pdf, delta, np.inf, args=(44,))[0]
            p_values.append(p_value)

        return p_values

    def cross_validation(self) -> (float, float):
        CVSS = []
        for i in range(self.n):
            temp_factors = np.delete(self.factors, i, 0)
            temp_answers = np.delete(self.answers, i, 0)
            temp_regression = LinearRegression(temp_factors, temp_answers)

            factor = self.factors[i]
            est_answer = factor @ temp_regression.coefficients
            CVSS.append((self.answers[i] - est_answer) ** 2)

        TSS = np.sum((self.answers - np.average(self.answers)) ** 2)
        cvss = np.sum(CVSS)
        return cvss, (TSS - cvss) / TSS

    def scale_coefficients(self):
        new_coefficients = []
        for coefficient in self.coefficients:
            new_coefficients.append((coefficient - np.min(self.coefficients)) /
                                    (np.max(self.coefficients) - np.min(
                                        self.coefficients)))
        return new_coefficients

    @property
    def n(self):
        return len(self.factors)

    @property
    def factors_count(self):
        return self.factors.shape[1]

    def __repr__(self):
        return str(self)

    def __str__(self):
        return (f'Коэффициенты регрессии: {list(self.coefficients)}' +
                f'\nRSS регрессии: {self.RSS}' +
                f'\nTSS регрессии: {self.TSS}' +
                f'\nR^2 регрессии: {self.R_square}' +
                f'\nF: {self.F}' +
                f'\nF^-1: {self.inv_F}' +
                f'\nРегрессия значима: {"да" if self.is_regression_significant() else "нет"}')

In [72]:
regression = LinearRegression(sample, Y)

In [73]:
regression

Коэффициенты регрессии: [array([84.]), array([85.5]), array([87.81818182]), array([90.]), array([91.])]
RSS регрессии: 23.136363636363633
TSS регрессии: 122.16
R^2 регрессии: 0.8106060606060607
F: [[ 2.  0.  0.  0.  0.]
 [ 0.  6.  0.  0.  0.]
 [ 0.  0. 11.  0.  0.]
 [ 0.  0.  0.  4.  0.]
 [ 0.  0.  0.  0.  2.]]
F^-1: [[ 0.5         0.          0.          0.          0.        ]
 [ 0.          0.16666667  0.          0.          0.        ]
 [ 0.          0.          0.09090909  0.          0.        ]
 [-0.         -0.         -0.          0.25       -0.        ]
 [ 0.          0.          0.          0.          0.5       ]]
Регрессия значима: да

## Попарное сравнение средних в рамках регрессионной модели, с учётом множественности проверяемых гипотез

In [74]:
def check_coefficients(_regression: LinearRegression) -> (list[list[int], ...], ...):
    deltas = [[] for _ in range(_regression.factors_count - 1)]
    p_values = [[] for _ in range(_regression.factors_count - 1)]

    for i in range(0, _regression.factors_count - 1):
        for j in range(i + 1, _regression.factors_count):
            betta_1 = _regression.coefficients[i]
            betta_2 = _regression.coefficients[j]
            RSS = _regression.RSS
            F_sum = _regression.inv_F[i][i] + _regression.inv_F[j][j]
            
            delta = (betta_1 - betta_2) / np.sqrt(RSS * F_sum) * np.sqrt(20)
            p_value = sps.t.cdf(delta, 20)
            deltas[i].append(delta[0])
            p_values[i].append(p_value[0])
    return p_values, deltas

In [76]:
p_values, deltas = check_coefficients(regression)
p_values = flatten(p_values)
deltas = flatten(deltas)

### Сортируем $p_{value}$ по возрастанию и начинаем проверять по методу Холла-Бонферони

In [84]:
p_values = np.sort(p_values)
for i, p_value in enumerate(p_values, 0):
    if p_value > 0.05 / (10 - i):
        print(i, p_value, 0.05 / (10 - i))

8 0.051550204450411884 0.025
9 0.1478956761373736 0.05


По методу Холла-Бонферони нет основания отвергать гипотезы